In [ ]:
import keras 

In [ ]:
from keras.models import Sequential
model = Sequential()

In [ ]:
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Dense(256, activation='relu'))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
model.add(Flatten())

In [ ]:
#Taking inputs

In [ ]:
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape =(224, 224, 3)))

In [ ]:
# 
model.compile(loss='binary_crossentropy', optimizer = 'rmsprop')

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X_train, Y_train, batch_size = 323, epochs=10,validation_data = (x_val, y_val))

NameError: ignored

In [ ]:
score = model.evaluate(x_test, y_test, batch_szie=32)

In [ ]:
model.save_weights("model.h5")
model.load_weights('model.h5')

In [ ]:
#Functional API

In [ ]:
from keras.models import Model

In [ ]:
from keras.layers import Input

In [ ]:
digit_input = Input(shape=(244, 244, 3))

In [ ]:
x=Conv2D(64,(3,3))(digit_input)
x=Conv2D(64,(3,3))(x)
x=MaxPooling2D((2,2))(x)
out = Flatten()(x)

In [ ]:
vision_model = Model(digit_input, out)

In [ ]:
vision_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
Total params: 37,568
Trainable params: 37,568
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os
os.listdir('.')

['.config', 'sample_data']

In [ ]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"
 
def fire_module(x, fire_id, squeeze=16, expand=64):
   s_id = 'fire' + str(fire_id) + '/'
   x = Conv2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
   x = Activation('relu', name=s_id + relu + sq1x1)(x)
 
   left = Conv2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
   left = Activation('relu', name=s_id + relu + exp1x1)(left)
 
   right = Conv2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
   right = Activation('relu', name=s_id + relu + exp3x3)(right)
 
   x = keras.layers.Concatenate(axis=3, name=s_id + 'concat')([left, right])
   return x

In [ ]:
x = Conv2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(digit_input)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

In [ ]:
x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)
 
x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)
 
x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)
x = Dropout(0.5, name='drop9')(x)

In [ ]:
x = Conv2D(5, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = keras.layers.GlobalAveragePooling2D()(x)
out = Activation('softmax', name='loss')(x)

In [ ]:

model = Model(digit_input, out, name='squeezenet')
model.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 121, 121, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 121, 121, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 60, 60, 64)   0           relu_conv1[0][0]                 
_________________________________________________________________________________________